# Generate Graphs from SODA 3.3.1 with SLIC ([Achanta et al., 2012](https://ieeexplore.ieee.org/abstract/document/6205760/?casa_token=TKLq-JwhmSkAAAAA:C1CdOBhAtt5EmjVE2gA0q-33KD60Rt8z3WO5wzO8vgzTz-1FnBWRicLOnKMvoW49urDoX6FJiJg))

by Ding

In [ ]:
import numpy as np
from netCDF4 import Dataset
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd

import torch

[Download](https://coastwatch.pfeg.noaa.gov/erddap/griddap/erdSoda331oceanmday.html) and upload SODA 3.3.1.

In [ ]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [ ]:
%%bash
cp -a "/gdrive/MyDrive/soda_331_pt_l5.nc" "/content/"

In [ ]:
soda = xr.open_dataset("soda_331_pt_l5.nc", decode_times=False)
soda

<xarray.Dataset>
Dimensions:    (time: 432, depth: 1, latitude: 330, longitude: 720)
Coordinates:
  * time       (time) float64 3.168e+08 3.195e+08 ... 1.448e+09 1.45e+09
  * depth      (depth) float32 5.034
  * latitude   (latitude) float32 -74.75 -74.25 -73.75 ... 88.75 89.25 89.75
  * longitude  (longitude) float32 0.25 0.75 1.25 1.75 ... 358.8 359.2 359.8
Data variables:
    temp       (time, depth, latitude, longitude) float32 ...
Attributes: (12/47)
    _CoordSysBuilder:              ucar.nc2.internal.dataset.conv.DefaultConv...
    acknowledgement:               The SODA3 project is funded by the US Nati...
    cdm_data_type:                 Grid
    Conventions:                   COARDS, CF-1.6, ACDD-1.3
    Created_by:                    Gennady Chepurin and Ligang Chen
    creation_date:                 October, 2016
    ...                            ...
    summary:                       SODA3.3.1 ocean state, forced by MERRA2. T...
    time_coverage_end:             2015-12-16T00:00:00Z
    time_coverage_start:           1980-01-16T00:00:00Z
    title:                         SODA 3.3.1 Ocean State, 1/2°, Global, 1980...
    Website:                       https://www.atmos.umd.edu/~ocean/
    Westernmost_Easting:           0.25

In [ ]:
soda_array = soda.to_array(dim="temp")
soda_array = soda_array.to_numpy()
soda_array = np.squeeze(soda_array, axis=(0,2))

In [ ]:
soda_array.shape
soda_array[0]

array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       ...,
       [-1.7895846, -1.78924  , -1.7888818, ..., -1.7922941, -1.7912207,
        -1.7901095],
       [-1.7790489, -1.7780232, -1.777153 , ..., -1.7808189, -1.7805831,
        -1.7801052],
       [-1.7726142, -1.772514 , -1.7724175, ..., -1.7726903, -1.7726848,
        -1.7726794]], dtype=float32)

Import the [repo](https://github.com/bknyaz/graph_attention_pool) for the paper: Attention over nodes in Graph Neural Networks using PyTorch ([Knyazev et al., 2019](https://arxiv.org/abs/1905.02850))

Use their example of evaluating a pretrained model on MNIST.

In [ ]:
!git clone https://github.com/bknyaz/graph_attention_pool.git
%cd graph_attention_pool

Cloning into 'graph_attention_pool'...
remote: Enumerating objects: 455, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 455 (delta 0), reused 0 (delta 0), pack-reused 452
Receiving objects: 100% (455/455), 119.93 MiB | 14.90 MiB/s, done.
Resolving deltas: 100% (253/253), done.
/content/graph_attention_pool


In [ ]:
import urllib.request
model_name = 'checkpoint_mnist-75sp_139255_epoch30_seed0000111.pth.tar'
model_url = 'https://github.com/bknyaz/graph_attention_pool/raw/master/checkpoints/%s' % model_name
model_path = 'checkpoints/%s' % model_name
urllib.request.urlretrieve(model_url, model_path)

('checkpoints/checkpoint_mnist-75sp_139255_epoch30_seed0000111.pth.tar',
 <http.client.HTTPMessage at 0x7fb02ee226d0>)

Load the model.

In [ ]:
from chebygin import ChebyGIN

state = torch.load(model_path)
args = state['args']
model = ChebyGIN(in_features=5, out_features=10, filters=args.filters, K=args.filter_scale,
                 n_hidden=args.n_hidden, aggregation=args.aggregation, dropout=args.dropout,
                 readout=args.readout, pool=args.pool, pool_arch=args.pool_arch)
model.load_state_dict(state['state_dict'])
model = model.eval()

ChebyGINLayer torch.Size([4, 20]) tensor([0.5665, 0.5960, 0.5305, 0.6250], grad_fn=<SliceBackward0>)
ChebyGINLayer torch.Size([64, 16]) tensor([0.5908, 0.5682, 0.6144, 0.5568, 0.4077, 0.5977, 0.4913, 0.5310, 0.5612,
        0.5420], grad_fn=<SliceBackward0>)
Default PyTorch init is used for layer torch.Size([1, 4]), std=0.242
p values [ 0.44642788 -0.04229611  0.35800374  0.48456722]
cos_sim -0.07371149957180023
ChebyGINLayer torch.Size([512, 256]) tensor([0.5995, 0.5726, 0.5823, 0.5854, 0.6019, 0.5394, 0.5892, 0.5748, 0.5760,
        0.5932], grad_fn=<SliceBackward0>)


Extract superpixels and create node features

In [ ]:
import scipy.ndimage
from skimage.segmentation import slic
from scipy.spatial.distance import cdist

# The number (n_segments) of superpixels returned by SLIC is usually smaller than requested, so we request more
superpixels = slic(soda_array[0], n_segments=95, compactness=0.25, multichannel=False)
sp_indices = np.unique(superpixels)
n_sp = len(sp_indices)  # should be 74 with these parameters of slic

sp_intensity = np.zeros((n_sp, 1), np.float32)
sp_coord = np.zeros((n_sp, 2), np.float32)  # row, col
for seg in sp_indices:
    mask = superpixels == seg
    sp_intensity[seg] = np.mean(soda_array[0][mask])
    sp_coord[seg] = np.array(scipy.ndimage.measurements.center_of_mass(mask))

# The model is invariant to the order of nodes in a graph
# We can shuffle nodes and obtain exactly the same results
ind = np.random.permutation(n_sp)
sp_coord = sp_coord[ind]
sp_intensity = sp_intensity[ind]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
  


Create edges between nodes in the form of adjacency matrix.

In [ ]:
sp_coord = sp_coord / soda_array.shape[1]
dist = cdist(sp_coord, sp_coord)  # distance between all pairs of nodes
sigma = 0.1 * np.pi  # width of a Guassian
A = np.exp(- dist / sigma ** 2)  # transform distance to spatial closeness
A[np.diag_indices_from(A)] = 0  # remove self-loops
A = torch.from_numpy(A).float().unsqueeze(0)

Prepare an input to the model and process it.

In [ ]:
N_nodes = sp_intensity.shape[0]
mask = torch.ones(1, N_nodes, dtype=torch.uint8)

# mean and std computed for superpixel features in the training set
mn = torch.tensor([0.11225057, 0.11225057, 0.11225057, 0.44206527, 0.43950436]).view(1, 1, -1)
sd = torch.tensor([0.2721889,  0.2721889,  0.2721889,  0.2987583,  0.30080357]).view(1, 1, -1)

node_features = (torch.from_numpy(np.pad(np.concatenate((sp_intensity, sp_coord), axis=1),
                                         ((0, 0), (2, 0)), 'edge')).unsqueeze(0) - mn) / sd    

y, other_outputs = model([node_features, A, mask, None, {'N_nodes': torch.zeros(1, 1) + N_nodes}])
alpha = other_outputs['alpha'][0].data